In [1]:

import pandas as pd
import re #regex library to format fast to read into pd 
import os

import matplotlib.pyplot as plt
import feather

import numpy as np
import platform
import mne


from mne.channels import find_ch_adjacency, make_1020_channel_selections
from mne.stats import spatio_temporal_cluster_test
from mne.stats import (ttest_1samp_no_p, bonferroni_correction, fdr_correction,


                       permutation_t_test, permutation_cluster_1samp_test,spatio_temporal_cluster_1samp_test)
import sys
if platform.system()=='Darwin':
    os.chdir('/Volumes/BBC/BBC/WP1/data/EEG/tsk/')
    #sys.path.append('/Users/leupinv/BBC/WP1/data/Code/python/BBC')
    #if this doesn't work pound line right above this, restart the kernel, rerun the cell. 
    #Then uncomment the same line and rerun

else:
    #os.chdir('Z:/BBC/WP1/data/EEG/tsk')
    #sys.path.append('C:/Users/Vivi/switchdrive/BBC/WP1/data/Code/python/BBC')
    os.chdir('Z:/BBC/WP1/data/EEG/tsk')

from base.files_in_out import getListOfFiles,GetFiles
from stats.stats_helper import filter_list, tTest_ana
import stats.t_tests_cluster as clus
import stats.stats_helper as s_hp
import base.base_constants as cs
from stats.stats_helper import filter_list, tTest_ana, filter_list_equal

C:\Users\Vivi\anaconda3\envs\mne\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.3, the latest is 0.5.5.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
C:\Users\Vivi\anaconda3\envs\mne\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
from  stats.t_tests_cluster import plot_from_BF


In [3]:
%load_ext autoreload
%autoreload 2
import matplotlib as mpl
mpl.rcParams['svg.fonttype']='none'
mpl.rcParams['text.usetex']=False
plt.style.use('default')

mpl.rcParams['svg.fonttype']='none'
mpl.rcParams['text.usetex']=False

In [4]:
def data_cutter(data, time_limits=[-0.3, 0.8], new_time_limits=[0.2, 0.6], sfreq=256):
    """
    Cuts a 3D EEG data array ( time x electrodes) based on specified time limits.
    Parameters:
        data (np.ndarray): EEG data of shape (electrodes, time)
        time_limits (list of float): Original epoch time range in seconds (e.g., [-0.3, 0.8])
        new_time_limits (list of float): Desired time range to extract (e.g., [0.2, 0.6])
        sfreq (int): Sampling frequency in Hz (default is 256)
    Returns:
        np.ndarray: The cut data with shape (new_time_points,electrodes)
    """
    # Calculate the sample indices corresponding to the new time limits
    start_idx = int(np.round((new_time_limits[0] - time_limits[0]) * sfreq))
    end_idx = int(np.round((new_time_limits[1] - time_limits[0]) * sfreq))
    # Cut the data along the time dimension (axis=1)
    return data[start_idx:end_idx,:]



# Plot tvalues and use BF as mask

# Card fit
## mass univariate figure, 3e-f-g

In [14]:
g_num='g01'
file_dir=f'ana/deconvolution/ep_betas/modfit/dc/{g_num}_modfit_hep_awareness_unaware_dc.ep'
f = open(file_dir)


In [15]:
data=np.loadtxt(file_dir)
data=data_cutter(data,time_limits=[-0.2,0.8],new_time_limits=[-0.1,0.7])

data.shape

(204, 128)

In [16]:
erp_type='hep'
modtype='cardfit'
subject_list=cs.G_N
X_sys=np.empty((len(subject_list),data.shape[1],data.shape[0]))
dc='dc'
for idx,g_num in enumerate(subject_list):
    file_dir=f'ana/deconvolution/ep_betas/cardfit/{dc}/{g_num}_sys_hep__Intercept__{dc}.ep'
    data=np.loadtxt(file_dir)
    data=data_cutter(data,time_limits=[-0.2,0.8],new_time_limits=[-0.1,0.7])



    X_sys[idx,:,:]=data.T




In [17]:
subject_list=cs.G_N
X_dia=np.empty((len(subject_list),data.shape[1],data.shape[0]))
for idx,g_num in enumerate(subject_list):
    file_dir=f'ana/deconvolution/ep_betas/cardfit/{dc}/{g_num}_dia_hep__Intercept__{dc}.ep'
    data=np.loadtxt(file_dir)
    data=data_cutter(data,time_limits=[-0.2,0.8],new_time_limits=[-0.1,0.7])

    X_dia[idx,:,:]=data.T




### load bf

In [18]:
cond="sys" # either sys or dia
dc_type=dc

#load corresponding file



file_dir=f'ana/deconvolution/stats/BF_matrix_{modtype}_{erp_type}_{cond}_{dc_type}.ep'
f = open(file_dir)

In [19]:
data_BF=np.loadtxt(file_dir)
data_BF=data_cutter(data_BF.T,time_limits=[-0.2,0.8],new_time_limits=[-0.1,0.7]).T
data_BF.shape

(128, 204)

In [20]:
X=X_sys-X_dia

In [21]:
%matplotlib qt
filepath=f'{modtype}_{erp_type}_{cond}_{dc_type}'
s_hp.get_tTest(X,crop_value=[-0.1,0.7],FDR=False,mask_BF=data_BF,BF=True, plot_times=[0.55],averages=0.05,png=filepath,topo_limits=[-9,9])

-0.1
number of time pointst: 25
length of minimum time points to be significant: 1
at the moment if only one TF is significant the electrode will be displayed as sig
(128, 204)


(array([[ 0.248365  ,  0.26676563,  0.35951501, ..., -0.95212936,
         -0.79109694, -0.68769476],
        [-0.04293515, -0.13859765, -0.18524604, ...,  0.64856484,
          0.86024254,  0.99243543],
        [-0.83707182, -0.88592267, -0.91539075, ...,  2.29017056,
          2.48816462,  2.55114392],
        ...,
        [-0.36067682, -0.55962702, -0.79300544, ...,  0.59800363,
          0.37129847,  0.1441203 ],
        [-0.42649205, -0.60635271, -0.72627958, ...,  0.70985018,
          0.60614368,  0.50389266],
        [-0.02085958, -0.01895672, -0.07689025, ...,  0.15262367,
          0.12418812,  0.1230762 ]]),
 array([[0.80560239, 0.79153743, 0.72181462, ..., 0.34890028, 0.43531008,
         0.49710891],
        [0.96604749, 0.89072607, 0.85432559, ..., 0.52172272, 0.39671162,
         0.32919419],
        [0.4093976 , 0.38294681, 0.36753588, ..., 0.02947327, 0.01883284,
         0.01627391],
        ...,
        [0.72095508, 0.58003101, 0.43421494, ..., 0.55447937, 0.71311416

# Compute gavgs

# waveform figure 3 a-b-c

In [4]:
g_num='g01'
file_dir=f'ana/deconvolution/ep_betas/cardfit/dc/{g_num}_sys_hep__Intercept__dc.ep'
f = open(file_dir)


In [5]:
data=np.loadtxt(file_dir)

data.shape

(256, 128)

In [12]:
subject_list=cs.G_N
X=np.empty((len(subject_list),data.shape[1],data.shape[0]))
sys_cond='unaware'
fit='awafit'
dc='nodc'
beta_type="_Intercept_"

for idx,g_num in enumerate(subject_list):
    file_dir=f'ana/deconvolution/ep_betas/{fit}/{dc}/{g_num}_{sys_cond}_hep_{beta_type}_{dc}.ep'
    data=np.loadtxt(file_dir)

    X[idx,:,:]=data.T




In [13]:
gavg=np.mean(X,0).T
filename=f'ana/deconvolution/ep_betas/{fit}/{dc}/gavg_{fit}_hep_{sys_cond}_{dc}.ep'

np.savetxt(filename,gavg)

# Plot waveforms gavgs


## plot cardfit sys vs dia

In [223]:

#define variables
modtype='cardfit'
erp_type="hep"
cond_1='sys'
dc_type_1="nodc"

k_1=f'{cond_1}_{dc_type_1}'
#load corresponding file


file_dir=f'ana/deconvolution/ep_betas/{modtype}/{dc_type_1}/gavg_{modtype}_{erp_type}_{cond_1}_{dc_type_1}.ep'
f = open(file_dir)

In [224]:
data_1=np.loadtxt(file_dir)
data_1=data_cutter(data_1,time_limits=[-0.2,0.8],new_time_limits=[-0.1,0.7])
data.shape

(204, 128)

In [225]:

#define variables

dc_type_2="nodc"
cond_2='dia'
k_2=f'{cond_2}_{dc_type_2}'

#load corresponding file


file_dir=f'ana/deconvolution/ep_betas/{modtype}/{dc_type_2}/gavg_{modtype}_{erp_type}_{cond_2}_{dc_type_2}.ep'
f = open(file_dir)

In [226]:
data_2=np.loadtxt(file_dir)
data_2=data_cutter(data_2,time_limits=[-0.2,0.8],new_time_limits=[-0.1,0.7])


In [227]:
def get_evo(data,tmin=-0.1):
    biosemi_montage = mne.channels.make_standard_montage('biosemi128')

    info = mne.create_info(ch_names=biosemi_montage.ch_names, sfreq=256.,
                           ch_types='eeg')
    evok=mne.EvokedArray(data,info,tmin=tmin)
    evok.set_montage(biosemi_montage)
    return evok

In [228]:
evo_1=get_evo(data_1.T)
evo_2=get_evo(data_2.T)

In [229]:
%matplotlib qt

evokeds={k_1:evo_1,k_2:evo_2}



In [233]:
for cond,evo in evokeds.items():
    if 'sys'in cond:
        color=['tab:orange']
    else:
        color=['#1f77b4']
    sub_dict={cond:evo}
    fig_fin=mne.viz.plot_compare_evokeds(sub_dict,ci=False,colors=color, picks=['A23'],show_sensors=True,legend='upper right',truncate_yaxis=False,combine='mean',ylim=dict(eeg=[-3*1e6,3*1e6]),vlines=[0,0.7],linestyles=['--'])
    file_dir=f'ana/deconvolution/figures/waveforms_gavg_{modtype}_{erp_type}_{cond}_oz.svg'
    fig_fin[0].savefig(file_dir,format='svg')


combining channels using "mean"


C:\Users\Vivi\AppData\Local\Temp\ipykernel_1836\257251769.py:7: RuntimeWarning: Only 1 channel in "picks"; cannot combine by method "mean".
  fig_fin=mne.viz.plot_compare_evokeds(sub_dict,ci=False,colors=color, picks=['A23'],show_sensors=True,legend='upper right',truncate_yaxis=False,combine='mean',ylim=dict(eeg=[-3*1e6,3*1e6]),vlines=[0,0.7],linestyles=['--'])


combining channels using "mean"


C:\Users\Vivi\AppData\Local\Temp\ipykernel_1836\257251769.py:7: RuntimeWarning: Only 1 channel in "picks"; cannot combine by method "mean".
  fig_fin=mne.viz.plot_compare_evokeds(sub_dict,ci=False,colors=color, picks=['A23'],show_sensors=True,legend='upper right',truncate_yaxis=False,combine='mean',ylim=dict(eeg=[-3*1e6,3*1e6]),vlines=[0,0.7],linestyles=['--'])


In [ ]:

fig_fin=mne.viz.plot_compare_evokeds(evokeds,ci=False, colors=color,picks=['A23'],show_sensors=True,legend='upper right',truncate_yaxis=False,combine='mean',ylim=dict(eeg=[-3*1e6,3*1e6]),vlines=[0,0.7],linestyles=['-','--','-','--'])

In [86]:
file_dir=f'ana/deconvolution/figures/waveforms_gavg_{modtype}_{erp_type}_{cond_1}{cond_2}_{dc_type_1}_oz.svg'
fig_fin[0].savefig(file_dir,format='svg')

In [131]:
file_dir=f'ana/deconvolution/figures/waveforms_gavg_{modtype}_{erp_type}_{cond_1}_dcnodc_oz.svg'
fig_fin[0].savefig(file_dir,format='svg')

In [61]:
file_dir=f'ana/deconvolution/figures/waveforms_gavg_{modtype}_{erp_type}_{cond}_dcnodc_oz.svg'
fig.savefig(file_dir,format='svg')

## plot fig3 a, basefit
simple intercept fit of all condition pooled, dc vs no dc

In [223]:

#define variables
modtype='base'
erp_type="hep"
cond_1='Intercept'
dc_type_1="dc"

k_1=f'{cond_1}_{dc_type_1}'
#load corresponding file


file_dir=f'ana/deconvolution/ep_betas/{modtype}/{dc_type_1}/gavg_{modtype}_{erp_type}_{cond_1}_{dc_type_1}.ep'
f = open(file_dir)

In [224]:
data_1=np.loadtxt(file_dir)
data_1=data_cutter(data_1,time_limits=[-0.2,0.8],new_time_limits=[-0.1,0.7])
data.shape

(204, 128)

In [225]:

#define variables

dc_type_2="nodc"
cond_2='Intercept'
k_2=f'{cond_2}_{dc_type_2}'

#load corresponding file


file_dir=f'ana/deconvolution/ep_betas/{modtype}/{dc_type_2}/gavg_{modtype}_{erp_type}_{cond_2}_{dc_type_2}.ep'
f = open(file_dir)

In [226]:
data_2=np.loadtxt(file_dir)
data_2=data_cutter(data_2,time_limits=[-0.2,0.8],new_time_limits=[-0.1,0.7])


In [227]:
def get_evo(data,tmin=-0.1):
    biosemi_montage = mne.channels.make_standard_montage('biosemi128')

    info = mne.create_info(ch_names=biosemi_montage.ch_names, sfreq=256.,
                           ch_types='eeg')
    evok=mne.EvokedArray(data,info,tmin=tmin)
    evok.set_montage(biosemi_montage)
    return evok

In [228]:
evo_1=get_evo(data_1.T)
evo_2=get_evo(data_2.T)

In [229]:
%matplotlib qt

evokeds={k_1:evo_1,k_2:evo_2}



In [154]:


evokeds=dict(dc=evo_1,nodc=evo_2)



fig_fin=mne.viz.plot_compare_evokeds(evokeds,ci=False, picks='A23',show_sensors=True,legend='upper right',truncate_yaxis=False,combine='mean',vlines=[0,0.49,0.52,0.7],ylim=dict(eeg=[-3*1e6,3*1e6]),linestyles=['-','--','-','--'],colors=['k','k'])


combining channels using "mean"
combining channels using "mean"


C:\Users\Vivi\AppData\Local\Temp\ipykernel_1836\2005466826.py:5: RuntimeWarning: Only 1 channel in "picks"; cannot combine by method "mean".
  fig_fin=mne.viz.plot_compare_evokeds(evokeds,ci=False, picks='A23',show_sensors=True,legend='upper right',truncate_yaxis=False,combine='mean',vlines=[0,0.49,0.52,0.7],ylim=dict(eeg=[-3*1e6,3*1e6]),linestyles=['-','--','-','--'],colors=['k','k'])


In [155]:
file_dir=f'ana/deconvolution/figures/waveforms_gavg_{modtype}_{erp_type}_{cond_1}_nodcdc.svg'
fig_fin[0].savefig(file_dir,format='svg')